### Install Packages and Import the same

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

### Get the Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### Sentiment Analyzer

In [3]:
tokens = tokenizer.encode("I hated this at worst , absolutely the Best.", return_tensors='pt')

In [4]:
result = model(tokens)

In [5]:
result.logits

tensor([[ 0.3237, -0.3555, -1.0177, -0.6527,  1.4966]],
       grad_fn=<AddmmBackward0>)

In [6]:
int(torch.argmax(result.logits))+1

5

### Scrap the Reviews from Web

In [50]:
reviews = []
r = requests.get('https://www.imdb.com/title/tt12844910/reviews?ref_=tt_urv')
soup = BeautifulSoup(r.text, 'html.parser')
this_content = soup.find_all('div', attrs={'class': 'content'})
for content in this_content:
    results = content.find('div', attrs = {'class': 'text'})
    reviews.append(results.text)

In [51]:
#content
reviews

["Bollywood should have realised by now that big stars and over the top action sequences without a proper story and screenplay doesn't make a good movie.Within few minutes into the movie we knew what we are getting into. Complete,mindless action sequences one after another. Not even SRK and Deepika could save this one.The dialogues were cheesy, and the stunts were over the top that after certain point it feels like a joke. I do love mindless action movies like fast and furious but this feels like a parody of all those filmsThis movie is over-ambitious for its own sake.Let's hope Bollywood atleast in the near future realise they need good scripts to a make a good movie .",
 'Yash Raj Films cashed on the \'Civilian Vs Military\' hysteria of their neighbouring country and produced a film which heavily relied on the charisma of ShahRukh Khan & massy cameo of superstar Salman Khan, who reprised his role from Tiger franchise.If you\'re looking for a depth or traditional spy genre movie, then

### Create a Dataframe to collect reviews and sentiment all at one place

In [52]:
import numpy as np
import pandas as pd

In [53]:
df = pd.DataFrame(np.array(reviews), columns = ['reviews'])

In [54]:
df

,reviews
0,Bollywood should have realised by now that big...
1,Yash Raj Films cashed on the 'Civilian Vs Mili...
2,Action packed entertainer and a visual treat f...
3,Nothing Orignal Every action scene is copied (...
4,Director Siddharth Anand has created the same ...
5,Directed by Siddharth Anand starring Shah Rukh...
6,All hype worth it after watching pathaan. Srk ...
7,"As usual Bollywood, cheap copy of Hollywood mo..."
8,"If you take a bond film , the bourne series an..."
9,"First things first, suspend your sense of disb..."


In [55]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [57]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

### Finally Display the DataFrame

In [58]:
df

,reviews,sentiment
0,Bollywood should have realised by now that big...,1
1,Yash Raj Films cashed on the 'Civilian Vs Mili...,2
2,Action packed entertainer and a visual treat f...,3
3,Nothing Orignal Every action scene is copied (...,2
4,Director Siddharth Anand has created the same ...,5
5,Directed by Siddharth Anand starring Shah Rukh...,4
6,All hype worth it after watching pathaan. Srk ...,4
7,"As usual Bollywood, cheap copy of Hollywood mo...",2
8,"If you take a bond film , the bourne series an...",5
9,"First things first, suspend your sense of disb...",1


#### The average Sentiment Score for all the reviews is

In [64]:
df['sentiment'].mean()


3.08